# Penalized (Lasso) logistic model

Remember with logistic regression, we are modelling an event occurrence e.g. someone defaults on their credit, someone buys a product etc.

We can also use logistic regression in a multi-class problem.

Let's illustrate that using the Wine data set:

https://scikit-learn.org/stable/datasets/toy_dataset.html#wine-dataset

This dataset consists of 178 instances and 13 variables, describing different wines and their characteristics. The dataset also gives us three classes

class_0
class_1
class_2

which we want to predict.

The class distribution is relatively evenly spread:

class_0 (59), class_1 (71), class_2 (48)

Remember that in some instances, when you're trying to predict particularly rare classes, you might need to do some pre-processing of the data (e.g. through oversampling). Otherwise, you run into the risk of the predictive model not being able to fit properly to those classes.

In [23]:
# We start by importing today's packages

import sklearn.datasets as datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as tts


# Load the dataset and specify which part will be the predictors and which the to-be-predicted outcome
dataset = datasets.load_wine()
X = dataset['data']
y = dataset['target']

# Split data, here 70/30 split
x_train, x_test, y_train, y_test = tts(X, y, test_size=0.3, random_state=42)


Remember that logistic regression (just like linear regression last week) is sensitive towards scale differences. We therefore scale our predictors. Output variables don't usually need scaling, as we are not making direct comparisons among them.

In [24]:
# Scaled predictors
x_train_scaled = StandardScaler().fit_transform(x_train)
x_test_scaled = StandardScaler().fit_transform(x_test)

In [25]:
import pandas as pd
import numpy as np


# Fit everything together nicely into one dataframe after scaling in the previous step.

data1 = pd.DataFrame(data= np.c_[dataset['data'], dataset['target']],
                     columns= dataset['feature_names'] + ['target'])                     

data1.describe()

# Note that the descriptives don't actually tell us anything about our target variable, because it's a label.
# You can see, however, that there doesn't seem to be any missing data (all counts 178) and you can also
# use these statistics to get an idea of the distribution of your predictors.

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258,0.938202
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474,0.775035
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000,0.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000,0.000000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000,1.000000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000,2.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000,2.000000


We now want to fit a logistic regression model to predict the membership in a group.

In this case, we choose a penalized regression model. You will remember that these models allow us to reduce the number of predictors (in this case 13 of them) to only contain the most informative ones.

For Lasso regression, the choice of the penalty factor is quite important. Thinking back to the lecture slides, this is your tuning parameter lambda. It describes how much you want to penalize for any added parameter.

- A lambda of 0 implies that all predictors should be kept and no penalty should be applied.
- A lambda of infinity would consider no predictors at all and a maximum penalty. Usually, we restrict the parameter to max out at 1 though.
- With an increasing lambda, we consider fewer predictors. This increases the bias of the model. High bias is related to underfitting your model.
- With a decreasing lambda, we consider more predictors. This increases the variance of the model. High variance is related to overfitting.

In order to choose an optimal lambda for our case, we run k-fold cross validation to tune it while accounting for the model accuracy.

In [26]:
from sklearn.linear_model import LogisticRegression

# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

from numpy import arange
from sklearn.model_selection import GridSearchCV

lasso_logistic_model = LogisticRegression(
    penalty='l1', # L1 penalty refers to Lasso regression (L2 would be ridge regression and 'elasticnet' elastic net)
    solver='liblinear') # we choose this solver here because it's the fastest for small datasets


# Let's now look for the optimal value of lambda:

grid = dict() 
grid['C'] = arange(0.0001, 1, 0.01)

# 5-fold cross validation, evaluating by model accuracy
search = GridSearchCV(lasso_logistic_model, grid, scoring='accuracy', cv=5, refit=True)
results = search.fit(x_train_scaled, y_train)

print('Config: %s' % results.best_params_)


Config: {'C': 0.4901}


In [27]:
results

GridSearchCV(cv=5,
             estimator=LogisticRegression(penalty='l1', solver='liblinear'),
             param_grid={'C': array([1.000e-04, 1.010e-02, 2.010e-02, 3.010e-02, 4.010e-02, 5.010e-02,
       6.010e-02, 7.010e-02, 8.010e-02, 9.010e-02, 1.001e-01, 1.101e-01,
       1.201e-01, 1.301e-01, 1.401e-01, 1.501e-01, 1.601e-01, 1.701e-01,
       1.801e-01, 1.901e-01, 2.001e-01, 2.101e-01, 2.201e-01, 2.301e-01,
       2.401e-...
       6.601e-01, 6.701e-01, 6.801e-01, 6.901e-01, 7.001e-01, 7.101e-01,
       7.201e-01, 7.301e-01, 7.401e-01, 7.501e-01, 7.601e-01, 7.701e-01,
       7.801e-01, 7.901e-01, 8.001e-01, 8.101e-01, 8.201e-01, 8.301e-01,
       8.401e-01, 8.501e-01, 8.601e-01, 8.701e-01, 8.801e-01, 8.901e-01,
       9.001e-01, 9.101e-01, 9.201e-01, 9.301e-01, 9.401e-01, 9.501e-01,
       9.601e-01, 9.701e-01, 9.801e-01, 9.901e-01])},
             scoring='accuracy')

We have identified our optimal lambda in this case to be 0.4901.

But what is the actual impact of this penalization term on the results of our logistic regression?


Let's visualize the coefficients of the lasso-logistic model and compare with those of the standard (non-penalized) model

In [28]:
# Coefficients of the lasso-logistic model
lasso_model = LogisticRegression(
    penalty='l1', # Lasso regression
    solver='liblinear',
    C = 0.4901).fit(x_train_scaled,y_train) #here is your new penalty term from before
print("Coefficients of the lasso logistic model: \n\n",lasso_model.coef_)

# Coefficients of the traditional logistic model
logistic_model = LogisticRegression( # use the same model
    penalty='none').fit(x_train_scaled,y_train) # but without the penalty
print("\n\n Coefficients of the traditional logistic model: \n\n",logistic_model.coef_)

Coefficients of the lasso logistic model: 

 [[ 1.03450196  0.          0.56434198 -1.04122607  0.          0.
   0.93442124  0.          0.          0.          0.          0.46094534
   1.64085113]
 [-1.4105756  -0.08080792 -0.80041383  0.46451271  0.          0.
   0.15585241  0.          0.29161264 -1.29587215  1.05645296  0.
  -1.42021967]
 [ 0.          0.17382347  0.15587845  0.          0.          0.
  -1.55535453  0.          0.          1.2964283  -0.79217463 -0.64661293
   0.        ]]


 Coefficients of the traditional logistic model: 

 [[ 4.07894388  1.40496807  2.6084901  -4.70856028  0.65505705  1.16054494
   2.13844972  0.65351972  1.11618752  0.3218668  -0.1976088   3.00237699
   4.26715433]
 [-6.59464113 -0.8653589  -5.73187583  4.49811726  0.26276841 -2.24041925
   3.08249089  0.82525615  1.7858793  -4.29955537  5.63507782 -0.32811002
  -6.42966757]
 [ 2.51569726 -0.53960917  3.12338573  0.21044302 -0.91782547  1.07987431
  -5.22094061 -1.47877587 -2.90206682  3.97

In [29]:
lasso_model.predict(x_test_scaled)

array([0, 0, 2, 0, 1, 0, 1, 2, 1, 2, 0, 2, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 2, 2, 2, 1, 1, 1, 0, 0, 1, 2, 0, 0, 0, 2, 2, 1, 2, 0, 1, 1, 1,
       2, 0, 1, 1, 2, 0, 1, 0, 0, 2])

Note how the output from the scikit learn package are not as "pretty" as the ones we got last week from our statsmodel summary. Indeed, scitkit learn outputs don't have a summary attribute, which means no p-values to look at!

In order to get those, we could use the statsmodel package. 

But today we're not that interested in p-value comparisons, so let's instead continue comparing predictions.

In [30]:
# Predictions with the lasso-logistic model
predictions_tuned_model = search.predict(x_test_scaled) # get predictions for our test data
# equivalent to: predictions_tuned_model = lasso_model.predict(x_test_scaled)
#
print("\n\n Predictions of the Lasso logistic model: \n\n", predictions_tuned_model)

# Predictions with the standard logistic model
logictic_model = LogisticRegression(solver='liblinear').fit(x_train_scaled,y_train)

predictions_non_tuned_model = logictic_model.predict(x_test_scaled)

print("\n\n Predictions of the traditional logistic model: \n\n", predictions_non_tuned_model)




 Predictions of the Lasso logistic model: 

 [0 0 2 0 1 0 1 2 1 2 0 2 0 1 0 1 1 1 0 1 0 1 1 2 2 2 1 1 1 0 0 1 2 0 0 0 2
 2 1 2 0 1 1 1 2 0 1 1 2 0 1 0 0 2]


 Predictions of the traditional logistic model: 

 [0 0 2 0 1 0 1 2 1 2 0 2 0 1 0 1 1 1 0 1 0 1 1 2 2 2 1 1 1 0 0 1 2 0 0 0 2
 2 1 2 0 1 1 2 2 0 1 1 2 0 1 0 0 2]


Let's evaluate the performance of each model by looking at their confusion matrices.

In problems with a small number of classes, they can give you a good overview of correctly and incorrectly classified elements.

We also compute the accuracy of both models based on the number of correctly/incorrectly classified instances.

In [31]:
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import accuracy_score as accuracy

print("Confusion matrix for the lasso-logistic model: \n"+str(cm(y_test,predictions_tuned_model)))

print("Confusion matrix for the traditional logistic model: \n"+str(cm(y_test,predictions_non_tuned_model)))

print("Accuracy lasso model: "+str(accuracy(y_test,predictions_tuned_model)))

print("Accuracy traditional model: "+str(accuracy(y_test,predictions_non_tuned_model)))

Confusion matrix for the lasso-logistic model: 
[[19  0  0]
 [ 0 21  0]
 [ 0  0 14]]
Confusion matrix for the traditional logistic model: 
[[19  0  0]
 [ 0 20  1]
 [ 0  0 14]]
Accuracy lasso model: 1.0
Accuracy traditional model: 0.9814814814814815


From these outputs, we can see that our tuned lasso logistic model outperforms the traditional one.

Discussion question: how significant should an improvement be to "justify" the additional work that has gone into the lasso regression model? Maybe there is a trade-off due to the additional computational costs?

# Addressing imbalance while tuning a classification model 

We have seen in our dataset above that the implementation of a (penalized) logistic regression is fairly straightforward in cases in which the datasets are well-balanced.

In many real-life scenarios you will encounter unbalanced datasets and you will have to do some pre-processing. Unbalanced datasets don't allow the algorithm to learn what determines group membership for all groups equally. This is a common problem in datasets where a specific class is rare, for example default in credit scoring or specific health conditions.

In this example, we will use a dataset on breast cancer. You will understand that being able to accurately predict breast cancer occurrence is very important - think about the danger of making different types of errors in this case:

- Someone does not have cancer, but is flagged as a positive case -> can cause distress
- Someone does have cancer, but is flagged as a negative case -> can cause bodily harm


In this exercise, we will use SMOTE (Synthetic Minority Oversampling Technique) to balance out classes and integrate it within the modelling cycle for tuning our lasso-logistic model.

In [32]:
# remove comment symbol from the following if getting error:

# !pip install imblearn


from imblearn.over_sampling import SMOTE

# https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html

from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.pipeline import Pipeline
import sklearn.datasets as datasets
from sklearn.model_selection import StratifiedKFold

# Load data set
dataset = datasets.load_breast_cancer()
X = dataset['data']
y = dataset['target']

# Split data
X_train, X_test, y_train, y_test = tts(X,y,test_size=0.3,stratify=y,random_state=11)

print(dataset.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

Have an initial look at the data. We have

- Number of Instances: 569
- Number of Attributes: 30 numeric

Note the class distribution:

- 212 - Malignant
- 357 - Benign

In [33]:
# Create a pipeline with SMOTE in it
pipeline = imbpipeline(steps = [['smote', SMOTE(random_state=11)],
                                ['scaler', StandardScaler()],
                                ['classifier', LogisticRegression(random_state=11, # this shuffles our data
                                                                  penalty='l1',
                                                                  solver='liblinear')]])

# The pipeline module from the imblearn package is very useful in this case. It allows us to automatically run
# multiple pre-processing and analysis steps in sequence.


# Grid for the shrinkage intensity, i.e. our penalty factor
grid = dict() 
grid['classifier__C'] = arange(0.0001, 2, 0.01)

# Setup stratified cross-validation
stratified_kfold = StratifiedKFold(n_splits=5,
                                   shuffle=True,
                                   random_state=11) # note that we are setting the same random seed for all modelling steps

# Run cross-validation using the AUC as scoring metric

search = GridSearchCV(pipeline, grid, scoring='roc_auc', cv=stratified_kfold, refit=True)
results = search.fit(X_train, y_train)
print('Config: %s' % results.best_params_)

# As a side note
# Remember that AUC is a great tool for comparison -between- models. It should NOT be used on its own for 
# model evaluation, it's strictly a comparison tool. Here, we use it to compare between cross-validation runs.


Config: {'classifier__C': 0.5201}


Our optimal lambda in this case is 0.5201.

Let's look at what this model has built us as a pipeline. Use this to double check the order of elements.

In [50]:
results

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=11, shuffle=True),
             estimator=Pipeline(steps=[('smote', SMOTE(random_state=11)),
                                       ('scaler', StandardScaler()),
                                       ['classifier',
                                        LogisticRegression(penalty='l1',
                                                           random_state=11,
                                                           solver='liblinear')]]),
             param_grid={'classifier__C': array([1.0000e-04, 1.0100e-02, 2.0100e-02, 3.0100e-02, 4.0100e-02,
       5.0100e-02, 6.01...
       1.7001e+00, 1.7101e+00, 1.7201e+00, 1.7301e+00, 1.7401e+00,
       1.7501e+00, 1.7601e+00, 1.7701e+00, 1.7801e+00, 1.7901e+00,
       1.8001e+00, 1.8101e+00, 1.8201e+00, 1.8301e+00, 1.8401e+00,
       1.8501e+00, 1.8601e+00, 1.8701e+00, 1.8801e+00, 1.8901e+00,
       1.9001e+00, 1.9101e+00, 1.9201e+00, 1.9301e+00, 1.9401e+00,
       1.9501e+00, 1.9601e+00, 1.9701e+00, 1.9801e+00, 1.9901e+00])},
             scoring='roc_auc')

In [35]:
cv_score = search.best_score_ # best AUC score over all cross-validation runs

test_score = search.score(X_test, y_test) # AUC score over test data

print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')

Cross-validation score: 0.9978344827586207
Test score: 0.981892523364486
